In [1]:
!pip install sklearn gensim nltk

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [5]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk
from concurrent.futures import ThreadPoolExecutor
nltk.download('punkt')

# 加载数据集
categories = None
newsgroups_data = fetch_20newsgroups(subset='all', categories=categories, remove=('headers', 'footers', 'quotes'))
data = pd.DataFrame({'text': newsgroups_data.data, 'label': newsgroups_data.target})

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['label'], test_size=0.2, random_state=42)

# 使用 TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# 训练逻辑回归模型使用 Tfidf 特征
model_tfidf = LogisticRegression()
model_tfidf.fit(X_train_tfidf, y_train)
predictions_tfidf = model_tfidf.predict(X_test_tfidf)
accuracy_tfidf = accuracy_score(y_test, predictions_tfidf)
print('Accuracy with Tfidf: ', accuracy_tfidf)

# 文本分词
X_train_tokenized = [word_tokenize(text.lower()) for text in X_train]
X_test_tokenized = [word_tokenize(text.lower()) for text in X_test]

# 训练 Word2Vec 模型
w2v_model = Word2Vec(sentences=X_train_tokenized, vector_size=100, window=5, min_count=1, workers=4)
w2v_model.train(X_train_tokenized, total_examples=len(X_train_tokenized), epochs=10)

# 将 Word2Vec 模型的词汇表转换为集合以加快查找速度
word_set = set(w2v_model.wv.index_to_key)

def document_vector(doc):
    doc = [word for word in doc if word in word_set]
    return np.mean(w2v_model.wv[doc], axis=0) if doc else np.zeros(100)

def compute_document_vectors(docs):
    with ThreadPoolExecutor(max_workers=4) as executor:
        return list(executor.map(document_vector, docs))

X_train_w2v = np.array(compute_document_vectors(X_train_tokenized))
X_test_w2v = np.array(compute_document_vectors(X_test_tokenized))

model_w2v = LogisticRegression()
model_w2v.fit(X_train_w2v, y_train)
predictions_w2v = model_w2v.predict(X_test_w2v)
accuracy_w2v = accuracy_score(y_test, predictions_w2v)
print('Accuracy with Word2Vec: ', accuracy_w2v)

# 比较两种方法的性能
print('Comparison of Models:')
print('TF-IDF Accuracy: ', accuracy_tfidf)
print('Word2Vec Accuracy: ', accuracy_w2v)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy with Tfidf:  0.6888594164456233


Accuracy with Word2Vec:  0.5084880636604775
Comparison of Models:
TF-IDF Accuracy:  0.6888594164456233
Word2Vec Accuracy:  0.5084880636604775


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
